In [1]:
%cd ..

/home/haito/kaggle/rsna-str-2/workdir


In [2]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.append(".")
from src.factory import *
from src.utils import *
from sklearn.metrics import log_loss

========== load_dicom_array() NOT USE TRY-EXCEPT ======================


In [3]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")


### train = train.query("fold == 0 or fold == 1")  # now I have fold0,1 only

studies = train.StudyInstanceUID.unique()

In [4]:
def get_pred(_path):
    res = load_pickle(_path)
    raw_pred = pd.DataFrame({
        "SOPInstanceUID": res["ids"],
        **res["outputs"],
    })
    return raw_pred

def calib_p(arr, factor):  # set factor>1 to enhance positive prob
    return arr * factor / (arr * factor + (1-arr))

In [5]:
# check
# oof_f0 = get_pred("output_yuji/b3_non_weight//oof_fold0_ep0.pkl")
# plt.hist( oof_f0.pe_present_on_image, bins=300 )
# plt.show()

In [6]:
oof_f0, fold0_calib_f = get_pred("output/final_image_level/oof_fold0.pkl"), 3.8250639579850194
oof_f1, fold1_calib_f = get_pred("output/final_image_level/oof_fold1.pkl"), 8.555037588568537
oof_f2, fold2_calib_f = get_pred("output/final_image_level/oof_fold2.pkl"), 4.374239635034443
oof_f3, fold3_calib_f = get_pred("output/final_image_level/oof_fold3.pkl"), 7.480972390526775
oof_f4, fold4_calib_f = get_pred("output/final_image_level/oof_fold4.pkl"), 5.002262078458348

# BAD
if False:  # pick best one which yields weighted-logloss after calib
    oof_f3, fold3_calib_f = get_pred("output/035_pe_present___448___apex/fold3_ep0.pt.valid.pickle"), 6.541753595870311
    oof_f4, fold4_calib_f = get_pred("output/035_pe_present___448___apex/fold4_ep0.pt.valid.pickle"), 3.8250639579850194

if True: ### ==== do calib for each fold
    oof_f0["pe_present_on_image"] = calib_p(oof_f0["pe_present_on_image"], fold0_calib_f)
    oof_f1["pe_present_on_image"] = calib_p(oof_f1["pe_present_on_image"], fold1_calib_f)
    oof_f2["pe_present_on_image"] = calib_p(oof_f2["pe_present_on_image"], fold2_calib_f)
    oof_f3["pe_present_on_image"] = calib_p(oof_f3["pe_present_on_image"], fold3_calib_f)
    oof_f4["pe_present_on_image"] = calib_p(oof_f4["pe_present_on_image"], fold4_calib_f)

oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred0'})

train = train.merge(oof[['pred0', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [7]:
train.head()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,...,true_filling_defect_not_pe,rightsided_pe,acute_and_chronic_pe,central_pe,indeterminate,exam_type,fold,pe_present_portion,z_pos,pred0
0,6897fa9de148,2bfbb7fd2e8b,c0f3cb036d06,0,0,0,0,0,0,1,...,0,1,0,0,0,positive,1,0.33871,123,0.002160
1,6897fa9de148,2bfbb7fd2e8b,f57ffd3883b6,0,0,0,0,0,0,1,...,0,1,0,0,0,positive,1,0.33871,114,0.000151
2,6897fa9de148,2bfbb7fd2e8b,41220fda34a3,0,0,0,0,0,0,1,...,0,1,0,0,0,positive,1,0.33871,24,0.218694
3,6897fa9de148,2bfbb7fd2e8b,13b685b4b14f,0,0,0,0,0,0,1,...,0,1,0,0,0,positive,1,0.33871,23,0.272379
4,6897fa9de148,2bfbb7fd2e8b,be0b7524ffb4,0,0,0,0,0,0,1,...,0,1,0,0,0,positive,1,0.33871,22,0.065160


In [8]:
train_copyed = train.copy()
train.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image', 'negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate',
       'exam_type', 'fold', 'pe_present_portion', 'z_pos', 'pred0'],
      dtype='object')

In [9]:
""" feature engineer """

' feature engineer '

In [10]:
train = train.sort_values(['StudyInstanceUID', 'z_pos'])
train_current_z_pos = train.groupby('StudyInstanceUID')['z_pos'].shift(0)

### for i in range(1, 20):
for i in range(1, 10):
    train[f'pred0_pre{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(i)
    train[f'pred0_post{i}'] = train.groupby('StudyInstanceUID')['pred0'].shift(-i)

# NORMALIZED Z POS
z_max = train.groupby('StudyInstanceUID').z_pos.max().rename('z_pos_max')
train = train.merge(z_max, on='StudyInstanceUID')
train['z_pos_norm'] = train['z_pos'] / train['z_pos_max']
train = train.drop('z_pos_max', axis=1)

In [11]:
train.tail()

,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,pe_present_on_image,negative_exam_for_pe,qa_motion,qa_contrast,flow_artifact,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1,...,pred0_post5,pred0_pre6,pred0_post6,pred0_pre7,pred0_post7,pred0_pre8,pred0_post8,pred0_pre9,pred0_post9,z_pos_norm
1790589,fffda3f22362,39ca5eaafffe,29e855db7f2b,0,1,0,0,0,0,0,...,NaN,1.633664e-06,NaN,1.558988e-06,NaN,3.526268e-07,NaN,9.371637e-07,NaN,0.976190
1790590,fffda3f22362,39ca5eaafffe,f7ca277a66c2,0,1,0,0,0,0,0,...,NaN,4.497674e-07,NaN,1.633664e-06,NaN,1.558988e-06,NaN,3.526268e-07,NaN,0.982143
1790591,fffda3f22362,39ca5eaafffe,59714fd8dd25,0,1,0,0,0,0,0,...,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.633664e-06,NaN,1.558988e-06,NaN,0.988095
1790592,fffda3f22362,39ca5eaafffe,b33567349fae,0,1,0,0,0,0,0,...,NaN,4.913005e-06,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.633664e-06,NaN,0.994048
1790593,fffda3f22362,39ca5eaafffe,53d378d07811,0,1,0,0,0,0,0,...,NaN,2.411817e-06,NaN,4.913005e-06,NaN,6.327988e-07,NaN,4.497674e-07,NaN,1.000000


In [12]:
ids = [c for c in list(train) if 'UID' in c]
targets = [
    'negative_exam_for_pe',
    'indeterminate',
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',
]
other_targets = [c for c in list(train) if 'pe_present_on_image' in c]
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + targets + ids + other_targets
### remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets
remove_cols = ['fold', 'path', 'weight', 'qa_contrast', 'qa_motion'] + ['exam_type','flow_artifact','pe_present_portion', 'true_filling_defect_not_pe'] + targets + ids + other_targets + ['z_pos']

features = sorted(list(set(list(train)) - set(remove_cols)))
print(features)

['pred0', 'pred0_post1', 'pred0_post2', 'pred0_post3', 'pred0_post4', 'pred0_post5', 'pred0_post6', 'pred0_post7', 'pred0_post8', 'pred0_post9', 'pred0_pre1', 'pred0_pre2', 'pred0_pre3', 'pred0_pre4', 'pred0_pre5', 'pred0_pre6', 'pred0_pre7', 'pred0_pre8', 'pred0_pre9', 'z_pos_norm']


In [13]:
features_copyed = features.copy()

In [14]:
def fobj(pred, data):
    true = data.get_label()
    label = 2*true - 1
    weights = data.weights
    response = -label / (1 + np.exp(label * pred))
    abs_response = np.abs(response)
    grad = response
    hess = abs_response * (1 - abs_response)
    return grad*weights, hess*weights

In [15]:
import torch
bce_func = torch.nn.BCELoss(reduction='none')

def feval2(preds, data):
    scores = bce_func(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [16]:
import torch
bce_func_logit = torch.nn.BCEWithLogitsLoss(reduction='none')

def feval(preds, data):
    scores = bce_func_logit(torch.FloatTensor(preds), torch.FloatTensor(data.label))
    scores = scores * torch.FloatTensor(data.weights)
    return 'weighted logloss', torch.mean(scores), False

In [17]:
import torch
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [18]:
# features = ['pred']  # for comparison
features = features_copyed

In [19]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
import warnings
warnings.simplefilter('ignore')
import pickle

# oof_preds_list = []
# models_list = []
# target = 'pe_present_on_image'

# for i in range(1):
#     print(f'=================={i}================')
#     if i % 4 == 0:
#         params = {'boosting_type': 'gbdt',
#             'objective': 'binary',
# #             'metric': 'None',
#             'subsample': 0.75,
#             'subsample_freq': 1,
#             'learning_rate': 0.1,
#             'feature_fraction': 0.9,
#             'max_depth': 15,
#             'lambda_l1': 1,  
#             'lambda_l2': 1,
#             'verbose': 100,
#             'early_stopping_rounds': 100,
#             'verbose': -1,
#             } 
#     elif i % 4 == 1:
#         params = {
#             'max_depth': 4,
#             'max_leave': int(0.2 * 2 ** 4),
#             'reg_lambda': 1,
#             'reg_alpha': 1,
#             'subsamples': 0.8,
#             'colsample_bytree': 0.7,
#             'objective': 'binary',
#             'min_data_in_leaf': 0,
#             'boosting': 'gbdt',
#             'metric': 'None',
#             'learning_rate': 0.1,
#                       }
#     elif i % 4 == 2:
#         params = {
#             'num_leaves': 19, 
#             'min_data_in_leaf': 160,
#             'min_child_weight': 0.03,
#             'bagging_fraction' : 0.7,
#             'feature_fraction' : 0.8,
#             'learning_rate' : 0.1,
#             'max_depth': -1,
#             'reg_alpha': 0.02,
#             'reg_lambda': 0.12,
#             'objective': 'binary',
#             'verbose': 100,
#             'boost_from_average': False,
#             'metric': 'None',
#         }  
#     else:
#         params = {
#             'objective': "binary",
#             'metric': 'None',
#             'boost_from_average': "false",
#             'tree_learner': "serial",
#             'max_depth': -1,
#             'learning_rate': 0.1,
#             'num_leaves': 197,
#             'feature_fraction': 0.3,
#             'bagging_freq': 1,
#             'bagging_fraction': 0.7,
#             'min_data_in_leaf': 100,
#             'bagging_seed': 11,
#             'max_bin': 255,
#             'verbosity': -1}    
        
#     oof_preds = np.zeros(train.shape[0])
#     val_results = {}
#     models = []
#     params['random_state'] = i
#     iter = 100000

#     for n_fold in range(5):
#     ### for n_fold in range(2):
#         print(f'    ==============fold{n_fold}================')
#         tr = train.query(f'fold != {n_fold}')
#         val = train.query(f'fold == {n_fold}')
#         trn_data = lgb.Dataset(tr[features], label=tr[target])
#         trn_data.weights = tr.pe_present_portion.values
#         val_data = lgb.Dataset(val[features], label=val[target])
#         val_data.weights = val.pe_present_portion.values
        
#         clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
# #                         verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
#                         feval=feval, fobj = fobj, verbose_eval=2000, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results, )
#         file = f'lgbs/lgb_seed{i}_fold{n_fold}.pkl'
#         pickle.dump(clf, open(file, 'wb'))
#         oof_preds[train.fold==n_fold] = clf.predict(val[features])
#     oof_preds_list.append(oof_preds)

# print(f'-------------------------------------------------------------------------roc_auc: {roc_auc_score(train[target], np.mean(oof_preds_list, axis=0))}')
# print(f'----------------------------------------------------------roc_auc using raw pred: {roc_auc_score(train[target], train["pred0"])}')
# print(f'------------------------------------------------------------------------------AP: {average_precision_score(train[target], np.mean(oof_preds_list, axis=0))}')
# print(f'---------------------------------------------------------------AP using raw pred: {average_precision_score(train[target], train["pred0"])}')

# lgb_oof = np.mean(oof_preds_list, axis=0)
# train['lgb_preds'] = sigmoid(lgb_oof)


==================0================
    ==============fold0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[53]	train's weighted logloss: 0.0110919	val's weighted logloss: 0.0110957
    ==============fold1================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	train's weighted logloss: 0.0107546	val's weighted logloss: 0.0116312
    ==============fold2================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > n

In [20]:
# bce_func = torch.nn.BCELoss(reduction='none')

# lgb_losses = bce_func(torch.FloatTensor(sigmoid(lgb_oof)), torch.FloatTensor(train['pe_present_on_image']))

# ### torch.mean(lgb_losses*train['weight'].values)
# torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.011420365194671893

In [21]:
# # no stacking result
# lgb_losses = bce_func(torch.FloatTensor(train['pred0']), torch.FloatTensor(train['pe_present_on_image']))
# torch.mean(lgb_losses*train['pe_present_portion'].values).item()

0.01322563879510154

In [133]:
# raise 

# <<< USED FOR FINAL SUB >>> stacking for exam-level targets

In [134]:
DATADIR = Path("../input/rsna-str-pulmonary-embolism-detection/")

train = pd.read_csv(DATADIR / "train.csv")

pre = pd.read_csv(DATADIR / "split.csv")
train = train.merge(pre, on="StudyInstanceUID")

portion = pd.read_csv(DATADIR / "study_pos_portion.csv")
train = train.merge(portion, on="StudyInstanceUID")

z_pos_df = pd.read_csv(DATADIR / "sop_to_prefix.csv").rename(columns={'img_prefix': 'z_pos'})
train = train.merge(z_pos_df, on="SOPInstanceUID")

studies = train.StudyInstanceUID.unique()

In [135]:
oof = pd.concat([oof_f0, oof_f1, oof_f2, oof_f3, oof_f4]).rename(columns={'pe_present_on_image': 'pred'})

train = train.merge(oof[['pred', 'SOPInstanceUID']], on="SOPInstanceUID")  # add pred

In [136]:
""" ADD MOANI """
if 1:  # after refactoring
    oof_monai = pd.concat([pd.read_csv(f"./output_jan/oof{fold}.csv") for fold in range(5)])
    oof_monai = oof_monai.drop(
        ['LOGITS_rv_lv_ratio_gte_1', 'LOGITS_central_pe', 'LOGITS_leftsided_pe', 
        'LOGITS_rightsided_pe', 'LOGITS_acute_and_chronic_pe', 'LOGITS_chronic_pe', 'fold'],
        axis=1)
else:  # used during comp
    oof_monai = pd.read_csv("output_jan/5foldmonai/monai_6targets_oof.csv").drop(
        ['LOGITS_rv_lv_ratio_gte_1', 'LOGITS_central_pe', 'LOGITS_leftsided_pe', 
        'LOGITS_rightsided_pe', 'LOGITS_acute_and_chronic_pe', 'LOGITS_chronic_pe', 'fold'],
        axis=1)

for key in oof_monai.columns:
    oof_monai = oof_monai.rename(columns={key: "monai_" + key})
oof_monai = oof_monai.rename(columns={"monai_study_id": "StudyInstanceUID"})
# print( oof_monai.head() )

# train = train.merge(oof_monai, on="StudyInstanceUID")  

In [137]:
# """ ADD chronics_b0 acute_bo """
# def load_5oof(model_dir):
#     oof = [ get_pred(model_dir + "/oof_fold%d.pkl" % fold ) for fold in range(5) ]
#     oof = pd.concat( oof ) #.rename(columns={})
#     oof_neg = [ get_pred(model_dir + "/oof_neg_fold%d.pkl" % fold ) for fold in range(5) ]
#     oof_neg = pd.concat( oof_neg ) #.rename(columns={})
#     return pd.concat([ oof, oof_neg ])

In [138]:
# USED RUDING COMP

# # oof_chronic = load_5oof( "output_yuji/chronics_b0" )
# # for k in oof_chronic.columns:
# #     if k == "SOPInstanceUID": continue
# #     oof_chronic = oof_chronic.rename(columns={k: "chronic_model_" + k})

# oof_position = load_5oof( "output_yuji/position_b0/" )
# for k in oof_position.columns:
#     if k == "SOPInstanceUID": continue
#     oof_position = oof_position.rename(columns={k: "position_model_" + k})

# print(len(train))
# # train = train.merge(oof_chronic , on='SOPInstanceUID')
# train = train.merge(oof_position, on='SOPInstanceUID')
# print(len(train))

1790594
1790594


In [ ]:
# AFTER REFACTORING
oof_position = pd.concat([ get_pred(f"output/final_position/oof_fold{fold}.pkl") for fold in range(5) ])

for k in oof_position.columns:
    if k == "SOPInstanceUID": continue
    oof_position = oof_position.rename(columns={k: "position_model_" + k})

print(len(train))
train = train.merge(oof_position, on='SOPInstanceUID')
print(len(train))

In [140]:
oof_position.columns

Index(['SOPInstanceUID', 'position_model_pe_present_on_image',
       'position_model_rightsided_pe', 'position_model_leftsided_pe',
       'position_model_central_pe'],
      dtype='object')

In [141]:
train.columns

Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SOPInstanceUID',
       'pe_present_on_image', 'negative_exam_for_pe', 'qa_motion',
       'qa_contrast', 'flow_artifact', 'rv_lv_ratio_gte_1', 'rv_lv_ratio_lt_1',
       'leftsided_pe', 'chronic_pe', 'true_filling_defect_not_pe',
       'rightsided_pe', 'acute_and_chronic_pe', 'central_pe', 'indeterminate',
       'exam_type', 'fold', 'pe_present_portion', 'z_pos', 'pred',
       'position_model_pe_present_on_image', 'position_model_rightsided_pe',
       'position_model_leftsided_pe', 'position_model_central_pe'],
      dtype='object')

In [142]:
from functools import partial
def grouping(df):
    grouped = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].mean())
    grouped = grouped.rename(columns={'pred': 'mean'})
    count = df.groupby('StudyInstanceUID')['pred'].count()
    grouped['count_total'] = count

    for i in range(1,10):
        count = df[df.pred>i/10].groupby('StudyInstanceUID')['pred'].count()
        grouped[f'count_over{i/10}'] = count
        grouped[f'count_over{i/10}_ratio'] = count / grouped['count_total']

    for q in [30, 50, 70, 80, 90, 95, 99]:
        grouped[f'percentile{q}'] = df.groupby('StudyInstanceUID')['pred'].apply(lambda arr: np.percentile(arr, q))

    ma = pd.DataFrame(df.groupby('StudyInstanceUID')['pred'].max())
    grouped['max'] = ma.pred

    grouped = grouped.reset_index().fillna(0)
    del grouped['count_total']
    return grouped

def grouping_one(df, pred_col):
    grouped = pd.DataFrame(df.groupby('StudyInstanceUID')[pred_col].mean())
    grouped = grouped.rename(columns={pred_col: f'{pred_col}_mean'})
    count = df.groupby('StudyInstanceUID')[pred_col].count()
    grouped['count_total'] = count

    for i in range(1,10):
        count = df[df[pred_col]>i/10].groupby('StudyInstanceUID')[pred_col].count()
        grouped[f'{pred_col}_count_over{i/10}'] = count
        grouped[f'{pred_col}_count_over{i/10}_ratio'] = count / grouped['count_total']
    count = df[df[pred_col]==1].groupby('StudyInstanceUID')[pred_col].count()

    for q in [30, 50, 70, 80, 90, 95, 99]:
        grouped[f'{pred_col}_percentile{q}'] = df.groupby('StudyInstanceUID')[pred_col].apply(lambda arr: np.percentile(arr, q))

    ma = pd.DataFrame(df.groupby('StudyInstanceUID')[pred_col].max())
    grouped[f'{pred_col}_max'] = ma[pred_col]

    grouped = grouped.reset_index().fillna(0)
    del grouped['count_total']
    return grouped


train_grouped = grouping(train)
# ADD MONAI
train_grouped = train_grouped.merge(oof_monai, on="StudyInstanceUID")

print( len (train_grouped) )
### ADD CHRONIC, POS
# for col in ['chronic_model_pe_present_on_image', 'chronic_model_chronic_pe', 'chronic_model_acute_and_chronic_pe', 'chronic_model_acute_pe'] \
#     + ['position_model_pe_present_on_image', 'position_model_rightsided_pe', 'position_model_leftsided_pe', 'position_model_central_pe']:
for col in ['position_model_pe_present_on_image', 'position_model_rightsided_pe', 'position_model_leftsided_pe', 'position_model_central_pe']:
    train_grouped = train_grouped.merge(
        grouping_one(train, col), on="StudyInstanceUID")


train_grouped['fold'] = train.groupby('StudyInstanceUID')['fold'].first().values
# add target
train_grouped['negative_exam_for_pe'] = train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values
train_grouped['positive_exam_for_pe'] = (1 - train.groupby('StudyInstanceUID')['negative_exam_for_pe'].first().values) * (1 - train.groupby('StudyInstanceUID')['indeterminate'].first().values)
for key in [
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',]:
    train_grouped[ key ] = train.groupby('StudyInstanceUID')[ key ].first().values

7279


In [143]:
print( list( train_grouped.columns ) )
train_grouped.head()

['StudyInstanceUID', 'mean', 'count_over0.1', 'count_over0.1_ratio', 'count_over0.2', 'count_over0.2_ratio', 'count_over0.3', 'count_over0.3_ratio', 'count_over0.4', 'count_over0.4_ratio', 'count_over0.5', 'count_over0.5_ratio', 'count_over0.6', 'count_over0.6_ratio', 'count_over0.7', 'count_over0.7_ratio', 'count_over0.8', 'count_over0.8_ratio', 'count_over0.9', 'count_over0.9_ratio', 'percentile30', 'percentile50', 'percentile70', 'percentile80', 'percentile90', 'percentile95', 'percentile99', 'max', 'monai_rv_lv_ratio_gte_1', 'monai_central_pe', 'monai_leftsided_pe', 'monai_rightsided_pe', 'monai_acute_and_chronic_pe', 'monai_chronic_pe', 'position_model_pe_present_on_image_mean', 'position_model_pe_present_on_image_count_over0.1', 'position_model_pe_present_on_image_count_over0.1_ratio', 'position_model_pe_present_on_image_count_over0.2', 'position_model_pe_present_on_image_count_over0.2_ratio', 'position_model_pe_present_on_image_count_over0.3', 'position_model_pe_present_on_image

,StudyInstanceUID,mean,count_over0.1,count_over0.1_ratio,count_over0.2,count_over0.2_ratio,count_over0.3,count_over0.3_ratio,count_over0.4,count_over0.4_ratio,...,fold,negative_exam_for_pe,positive_exam_for_pe,chronic_pe,acute_and_chronic_pe,central_pe,leftsided_pe,rightsided_pe,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1
0,0003b3d648eb,0.271391,121.0,0.542601,94.0,0.421525,80.0,0.358744,61.0,0.273543,...,0,1,0,0,0,0,0,0,0,0
1,000f7f114264,0.091386,65.0,0.271967,45.0,0.188285,28.0,0.117155,16.0,0.066946,...,3,1,0,0,0,0,0,0,0,0
2,00102474a2db,0.055245,58.0,0.177914,21.0,0.064417,10.0,0.030675,4.0,0.012270,...,3,1,0,0,0,0,0,0,0,0
3,0038fd5f09f5,0.054540,34.0,0.147826,13.0,0.056522,6.0,0.026087,6.0,0.026087,...,4,1,0,0,0,0,0,0,0,0
4,0045f113e031,0.133363,111.0,0.431907,75.0,0.291829,45.0,0.175097,22.0,0.085603,...,4,1,0,0,0,0,0,0,0,0


In [144]:
train_grouped

,StudyInstanceUID,mean,count_over0.1,count_over0.1_ratio,count_over0.2,count_over0.2_ratio,count_over0.3,count_over0.3_ratio,count_over0.4,count_over0.4_ratio,...,fold,negative_exam_for_pe,positive_exam_for_pe,chronic_pe,acute_and_chronic_pe,central_pe,leftsided_pe,rightsided_pe,rv_lv_ratio_gte_1,rv_lv_ratio_lt_1
0,0003b3d648eb,0.271391,121.0,0.542601,94.0,0.421525,80.0,0.358744,61.0,0.273543,...,0,1,0,0,0,0,0,0,0,0
1,000f7f114264,0.091386,65.0,0.271967,45.0,0.188285,28.0,0.117155,16.0,0.066946,...,3,1,0,0,0,0,0,0,0,0
2,00102474a2db,0.055245,58.0,0.177914,21.0,0.064417,10.0,0.030675,4.0,0.012270,...,3,1,0,0,0,0,0,0,0,0
3,0038fd5f09f5,0.054540,34.0,0.147826,13.0,0.056522,6.0,0.026087,6.0,0.026087,...,4,1,0,0,0,0,0,0,0,0
4,0045f113e031,0.133363,111.0,0.431907,75.0,0.291829,45.0,0.175097,22.0,0.085603,...,4,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7274,ffe5280cd21d,0.333355,113.0,0.470833,101.0,0.420833,89.0,0.370833,82.0,0.341667,...,4,0,1,0,0,0,0,1,0,1
7275,ffe7efaccbb9,0.018265,9.0,0.036000,3.0,0.012000,0.0,0.000000,0.0,0.000000,...,3,1,0,0,0,0,0,0,0,0
7276,fff1ef450040,0.028014,13.0,0.047794,6.0,0.022059,3.0,0.011029,3.0,0.011029,...,3,1,0,0,0,0,0,0,0,0
7277,fff9536823b1,0.181569,125.0,0.322165,85.0,0.219072,74.0,0.190722,71.0,0.182990,...,0,0,1,0,0,0,1,1,0,1


In [145]:

# features = list(set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']) - set(['count_total']))

### features = list( set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']) - set(['count_total']) - set( [
features = list( set(list(train_grouped)) - set(['StudyInstanceUID', 'positive_exam_for_pe', 'negative_exam_for_pe', 'fold']) - set( [
    'chronic_pe',
    'acute_and_chronic_pe',
    'central_pe',
    'leftsided_pe',
    'rightsided_pe',
    'rv_lv_ratio_gte_1',
    'rv_lv_ratio_lt_1',]  ) )
features = sorted(features)

### target = 'positive_exam_for_pe'

targets_monai = [
        'rv_lv_ratio_gte_1', # exam level
        "central_pe",
        "leftsided_pe",
        "rightsided_pe",
        "acute_and_chronic_pe",
        "chronic_pe"
    ]
#target = targets_monai[ 1 ]
target = "rv_lv_ratio_lt_1"

print("target", target)

target rv_lv_ratio_lt_1


In [146]:
print(features)

['count_over0.1', 'count_over0.1_ratio', 'count_over0.2', 'count_over0.2_ratio', 'count_over0.3', 'count_over0.3_ratio', 'count_over0.4', 'count_over0.4_ratio', 'count_over0.5', 'count_over0.5_ratio', 'count_over0.6', 'count_over0.6_ratio', 'count_over0.7', 'count_over0.7_ratio', 'count_over0.8', 'count_over0.8_ratio', 'count_over0.9', 'count_over0.9_ratio', 'max', 'mean', 'monai_acute_and_chronic_pe', 'monai_central_pe', 'monai_chronic_pe', 'monai_leftsided_pe', 'monai_rightsided_pe', 'monai_rv_lv_ratio_gte_1', 'percentile30', 'percentile50', 'percentile70', 'percentile80', 'percentile90', 'percentile95', 'percentile99', 'position_model_central_pe_count_over0.1', 'position_model_central_pe_count_over0.1_ratio', 'position_model_central_pe_count_over0.2', 'position_model_central_pe_count_over0.2_ratio', 'position_model_central_pe_count_over0.3', 'position_model_central_pe_count_over0.3_ratio', 'position_model_central_pe_count_over0.4', 'position_model_central_pe_count_over0.4_ratio', 'p

In [147]:
print(len(features))

141


In [148]:
""" WITH MONAI FEATURE """

import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter('ignore')
import pickle
from sklearn.model_selection import KFold, StratifiedKFold

### for target in targets_monai + ['rv_lv_ratio_lt_1']:
### for target in ['positive_exam_for_pe']:
for target in targets_monai + ['rv_lv_ratio_lt_1'] + ['positive_exam_for_pe']:

    print(f"~~~~~~~~~~~~~~~~~~~~~ {target} ~~~~~~~~~~~~~~~~~ \n")



    oof_preds_list = []
    test_preds_list = []
    models_list = []

    for i in range(1):
        print(f'=================={i}================')
        params = {'boosting_type': 'gbdt',
            'objective': 'binary',
    #             'metric': 'None',
            'subsample': 0.75,
            'subsample_freq': 1,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'max_depth': 15,
            'lambda_l1': 1,  
            'lambda_l2': 1,
            'verbose': 100,
            'early_stopping_rounds': 100,
            'verbose': -1,
            } 
            
        oof_preds = np.zeros(train_grouped.shape[0])

        ### test_preds = np.zeros(test_grouped.shape[0])
        val_results = {}
        models = []
        params['random_state'] = i
        iter = 100000
        kf = KFold(n_splits=5, shuffle=True, random_state=72)

        for n_fold in range( 5 ):
            tr = train_grouped[train_grouped.fold != n_fold]
            val = train_grouped[train_grouped.fold == n_fold]
            trn_data = lgb.Dataset(tr[features], label=tr[target])
            val_data = lgb.Dataset(val[features], label=val[target])
            
            clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
    #                         feval=feval, verbose_eval=10, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
                            verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
            file = f'lgbs/{target}_monai_lgb_seed{i}_fold{n_fold}.pkl'
            pickle.dump(clf, open(file, 'wb'))
            models.append(clf)
            
            oof_preds[train_grouped.fold == n_fold] = clf.predict(val[features])
        oof_preds_list.append(oof_preds)

    print(f'--------------------------------------------------------------------roc: {roc_auc_score(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
    print(f'----------------------------------------------------------------logloss: {log_loss(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
    lgb_oof_exam = np.mean(oof_preds_list, axis=0)


~~~~~~~~~~~~~~~~~~~~~ positive_exam_for_pe ~~~~~~~~~~~~~~~~~ 

==================0================
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	train's binary_logloss: 0.187879	val's binary_logloss: 0.309329
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	train's binary_logloss: 0.239731	val's binary_logloss: 0.318527
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[32]	train's binary_logloss: 0.226629	val's binary_logloss: 0.315525
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_le

In [149]:
# """WITHOUT MONAI"""

# target = "positive_exam_for_pe"

# features_without_monai = [e for e in features if not e.startswith("monai_")]
# print( features_without_monai )
# features = sorted( features_without_monai )

# import lightgbm as lgb
# import numpy as np
# from sklearn.metrics import roc_auc_score
# import warnings
# warnings.simplefilter('ignore')
# import pickle
# from sklearn.model_selection import KFold, StratifiedKFold

# oof_preds_list = []
# test_preds_list = []
# models_list = []

# for i in range(1):
#     print(f'=================={i}================')
#     params = {'boosting_type': 'gbdt',
#         'objective': 'binary',
# #             'metric': 'None',
#         'subsample': 0.75,
#         'subsample_freq': 1,
#         'learning_rate': 0.1,
#         'feature_fraction': 0.9,
#         'max_depth': 15,
#         'lambda_l1': 1,  
#         'lambda_l2': 1,
#         'verbose': 100,
#         'early_stopping_rounds': 100,
#         'verbose': -1,
#         } 
        
#     oof_preds = np.zeros(train_grouped.shape[0])

#     ### test_preds = np.zeros(test_grouped.shape[0])
#     val_results = {}
#     models = []
#     params['random_state'] = i
#     iter = 100000
#     kf = KFold(n_splits=5, shuffle=True, random_state=72)

#     for n_fold in range( 5 ):
#         tr = train_grouped[train_grouped.fold != n_fold]
#         val = train_grouped[train_grouped.fold == n_fold]
#         trn_data = lgb.Dataset(tr[features_without_monai], label=tr[target])
#         val_data = lgb.Dataset(val[features_without_monai], label=val[target])
        
#         clf = lgb.train(params, trn_data, num_boost_round=iter, valid_sets=[trn_data, val_data], valid_names=['train', 'val'],
# #                         feval=feval, verbose_eval=10, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
#                         verbose_eval=200, early_stopping_rounds = 10/params['learning_rate'], evals_result=val_results,)
#         file = f'lgbs/posexam_lgb_seed{i}_fold{n_fold}.pkl'
#         pickle.dump(clf, open(file, 'wb'))
#         models.append(clf)
        
#         oof_preds[train_grouped.fold == n_fold] = clf.predict(val[features_without_monai])
#     oof_preds_list.append(oof_preds)

# print(f'--------------------------------------------------------------------roc: {roc_auc_score(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
# print(f'----------------------------------------------------------------logloss: {log_loss(train_grouped[target], np.mean(oof_preds_list, axis=0))}')
# lgb_oof_exam = np.mean(oof_preds_list, axis=0)


In [150]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(torch.FloatTensor(oof_preds), torch.FloatTensor(train_grouped['positive_exam_for_pe']))

torch.mean(lgb_losses).item()

0.32274290919303894

In [151]:
bce_func = torch.nn.BCELoss(reduction='mean')
lgb_losses = bce_func(
    ( 1 - torch.FloatTensor(oof_preds) ) * (4911) / (4911 + 157), 
    torch.FloatTensor(train_grouped['negative_exam_for_pe']))

torch.mean(lgb_losses).item()

0.35468363761901855

In [152]:
# # current yama's pipeline for fold0-ep1
# def calib_p(arr, factor):  # set factor>1 to enhance positive prob
#     return arr * factor / (arr * factor + (1-arr))
# def post_yama(arr):
#     return calib_p( np.percentile(arr, 95), factor=1/8.5550)

# lgb_losses = bce_func(torch.FloatTensor(train[['StudyInstanceUID','pred']].groupby('StudyInstanceUID').apply(post_yama)), torch.FloatTensor(train_grouped['positive_exam_for_pe'])).item()
# lgb_losses